In [2]:
df = pd.read_csv('data/merged_data_brand_dropna.csv')

In [4]:
import pandas as pd

# 1. 날짜 컬럼 생성
df['event_date'] = pd.to_datetime(df['event_time_moscow']).dt.date

# 2. 퍼널 단계 정의
funnel_steps = ['view', 'cart', 'purchase']

In [6]:
# 1. 일자+유저+event_type별 카운트 (0/1)
df['flag'] = 1
daily_funnel = (
    df.drop_duplicates(['event_date', 'user_id', 'event_type'])  # 중복 제거
      .pivot_table(
          index=['event_date', 'user_id'],
          columns='event_type',
          values='flag',
          aggfunc='max',  # 1 또는 NaN
          fill_value=0
      )
      .reset_index()
)

# 2. 필요한 컬럼 순서 고정
daily_funnel = daily_funnel[['event_date', 'user_id'] + funnel_steps]

In [8]:
daily_funnel[daily_funnel['purchase'] == 1]

event_type,event_date,user_id,view,cart,purchase
11,2019-10-01,75972882,1,1,1
22,2019-10-01,128562720,1,1,1
23,2019-10-01,142717569,0,0,1
28,2019-10-01,151045021,1,1,1
37,2019-10-01,171204327,1,1,1
...,...,...,...,...,...
2048811,2020-02-29,622041514,1,1,1
2048838,2020-02-29,622042698,0,1,1
2048841,2020-02-29,622042844,1,1,1
2048907,2020-02-29,622046885,1,1,1


In [10]:
# 4. CSV 저장
daily_funnel.to_csv('data/001_daily_user_funnel.csv', index=False)